## Data Set and Optimization Solver

In [1]:
import pyepo

Auto-Sklearn cannot be imported.


In [2]:
# generate data
grid = (2,2) # grid size
num_data = 1000 # number of training data
num_feat = 5 # size of feature
deg = 4 # polynomial degree
e = 0.5 # noise width
feats, costs = pyepo.data.shortestpath.genData(num_data+1000, num_feat, grid, deg, e, seed=42)

In [3]:
from pyepo.model.grb import shortestPathModel
# set solver
optmodel = shortestPathModel(grid)
# test
optmodel.setObj(costs[0])
sol, obj = optmodel.solve()
print("Obj: {}".format(obj))
for i, e in enumerate(optmodel.arcs):
    if sol[i] > 1e-3:
        print(e)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-01
Obj: 0.5009689961416153
(0, 2)
(2, 3)


In [4]:
# split data
from sklearn.model_selection import train_test_split
x_train, x_test, c_train, c_test = train_test_split(feats, costs, test_size=1000, random_state=42)
# get training and test data set
dataset_train = pyepo.data.dataset.optDataset(optmodel, x_train, c_train)
dataset_test = pyepo.data.dataset.optDataset(optmodel, x_test, c_test)

Optimizing for optDataset...


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 5431.76it/s]


Optimizing for optDataset...


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 5463.62it/s]


In [5]:
# get data loader
from torch.utils.data import DataLoader
batch_size = 32
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

## Prediction Model

In [6]:
import torch
from torch import nn

# build linear model
class LinearRegression(nn.Module):
    
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(num_feat, (grid[0]-1)*grid[1]+(grid[1]-1)*grid[0])
    
    def forward(self, x):
        out = self.linear(x)
        return out

In [7]:
# init model
reg = LinearRegression()

## Loss 

In [8]:
from torch.autograd import Function

from pyepo import EPO

In [9]:
class polarConeAngle(nn.Module):
    """
    A autograd module for polar cone fitting loss with binary variables
    """
    
    def forward(self, pred_cost, true_sol):
        """
        Forward pass
        """
        loss = polarConeAngleFunc.apply(pred_cost, true_sol)
        return loss
    
class polarConeAngleFunc(Function):
    """
    A autograd function for polar cone fitting loss with binary variables
    """
    
    @staticmethod
    def forward(ctx, pred_cost, true_sol):
        loss = []
        for i in range(len(pred_cost)):
            # check if the vector in the cone
            if optmodel.modelSense == EPO.MINIMIZE:
                inside = torch.equal(pred_cost[i] <= 0, true_sol[i].type(torch.BoolTensor))
            if optmodel.modelSense == EPO.MAXIMIZE:
                inside = torch.equal(pred_cost[i] >= 0, true_sol[i].type(torch.BoolTensor))
            # z
            if inside:
                loss.append(0)
            else:
                

In [10]:
# init loss
pca_loss = polarConeAngle()

## Warning: Numerical Issue for Arcos

In [11]:
torch.acos(torch.tensor([0, 1, 1+1e8]))

tensor([1.5708, 0.0000,    nan])

## Test 

In [13]:
for data in loader_train:
    x, c, w, z = data
    # forward pass
    cp = reg(x)
    loss = pca_loss(cp, w)

[0.0, 1.0, 0.0, 1.0]
tensor([0., 1., 0., 1.])
[0.0, 1.0, 0.0, 1.0]
tensor([0., 1., 0., 1.])
[0.0, 1.0, 0.0, 1.0]
tensor([0., 1., 0., 1.])
[0.0, 1.0, 0.0, 1.0]
tensor([0., 1., 0., 1.])
[0.0, 1.0, 0.0, 1.0]
tensor([0., 1., 0., 1.])
[0.0, 1.0, 0.0, 1.0]
tensor([0., 1., 0., 1.])
[0.0, 1.0, 0.0, 1.0]
tensor([0., 1., 0., 1.])
[0.0, 1.0, 0.0, 1.0]
tensor([0., 1., 0., 1.])
